In [1]:

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

2024-01-19 18:06:18.333222: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-19 18:06:18.354470: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 18:06:18.354491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 18:06:18.355051: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 18:06:18.358669: I tensorflow/core/platform/cpu_feature_guar

In [2]:
x = pd.read_csv("Cary_Weather_Data.csv")
x['DATE'] = pd.to_datetime(x[['YEAR', 'MONTH', 'DAY']])
x = x.drop(columns=['YEAR','MONTH','DAY'])
#x = x.drop(columns=['Humidity','Precipitation','Wind Speed'])
x = x[['DATE', 'Temperature', 'Humidity', 'Precipitation', 'Wind Speed']]
x.head()
x.set_index('DATE', inplace=True)
x.head()


,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,15.12,10.68,6.65,3.59
1981-07-02,7.53,13.12,10.68,1.64
1981-07-03,7.32,13.79,37.42,2.43
1981-07-04,10.43,14.40,8.41,2.29
1981-07-05,13.13,15.20,4.13,1.58


In [3]:
scaler = StandardScaler()
x_scale = scaler.fit_transform(x)
x_scale = pd.DataFrame(x_scale, columns=x.columns, index=x.index)
#x_scale = x_scale.drop(columns=['Humidity', 'Precipitation', 'Wind Speed'])
x_scale.head()

,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,0.926990,0.385333,0.478536,1.918945
1981-07-02,-1.305674,0.925994,1.044274,-0.352127
1981-07-03,-1.367448,1.074455,4.798080,0.567948
1981-07-04,-0.452614,1.209620,0.725608,0.404897
1981-07-05,0.341614,1.386887,0.124774,-0.422006


In [4]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [5]:
time_steps = 1

X, y = create_dataset(x_scale, x_scale['Temperature'], time_steps)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)
X_train.shape
X_val.shape

(3106, 1, 4)

In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(filters=512, kernel_size=2, strides=1, padding='causal', activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(tf.keras.layers.LSTM(256, return_sequences=True))
model.add(tf.keras.layers.LSTM(128, return_sequences=False))
model.add(tf.keras.layers.Dense(1))
#model.compile(loss='mse', optimizer='adam', metrics=['rmse'])
model.summary()

2024-01-19 18:06:19.895598: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 18:06:19.920208: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 18:06:19.925198: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1, 512)            4608      
                                                                 
 lstm (LSTM)                 (None, 1, 256)            787456    
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 989313 (3.77 MB)
Trainable params: 989313 (3.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [8]:
cp = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_mae')
early_stoping = tf.keras.callbacks.EarlyStopping(monitor='mae', restore_best_weights=True, patience=30)

In [9]:

history = model.fit(X_train, y_train, batch_size=32 , epochs=100, validation_data=(X_val, y_val), callbacks=[cp, early_stoping], shuffle=False)

Epoch 1/100


2024-01-19 18:06:21.641439: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-19 18:06:21.692119: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


  1/389 [..............................] - ETA: 10:57 - loss: 0.4939 - mae: 0.5829

2024-01-19 18:06:22.020997: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fea5967d330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-19 18:06:22.021019: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-01-19 18:06:22.023825: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1705705582.067035   23117 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


389/389 [==============================] - 4s 5ms/step - loss: 0.8084 - mae: 0.7110 - val_loss: 0.7631 - val_mae: 0.6979
Epoch 2/100
389/389 [==============================] - 2s 5ms/step - loss: 0.7886 - mae: 0.7014 - val_loss: 0.7589 - val_mae: 0.6957
Epoch 3/100
389/389 [==============================] - 2s 4ms/step - loss: 0.7843 - mae: 0.6993 - val_loss: 0.7571 - val_mae: 0.6949
Epoch 4/100
389/389 [==============================] - 2s 4ms/step - loss: 0.7814 - mae: 0.6977 - val_loss: 0.7562 - val_mae: 0.6944
Epoch 5/100
389/389 [==============================] - 2s 4ms/step - loss: 0.7792 - mae: 0.6965 - val_loss: 0.7558 - val_mae: 0.6942
Epoch 6/100
389/389 [==============================] - 2s 6ms/step - loss: 0.7773 - mae: 0.6956 - val_loss: 0.7555 - val_mae: 0.6940
Epoch 7/100
389/389 [==============================] - 3s 7ms/step - loss: 0.7757 - mae: 0.6947 - val_loss: 0.7552 - val_mae: 0.6940
Epoch 8/100
389/389 [==============================] - 2s 5ms/step - loss: 0.7744

In [12]:

matplotlib.interactive(True)
%matplotlib qt
plt.plot(history.history['val_mae'], label='Validation MAE')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('val mae')
plt.legend()
plt.show()

 1/98 [..............................] - ETA: 33s

In [13]:
prediction = model.predict(X_val)
prediction_reshape = prediction[:, 0]
x_val_reshz = X_val[:, 0, 0]

98/98 [==============================] - 0s 977us/step


In [14]:
time_val = x.index[-len(X_val):]

In [15]:
plt.plot(x_scale['Temperature'], color='green')
plt.plot(time_val, x_val_reshz, label='Actual', color='blue')
plt.plot(time_val, prediction_reshape, label='Prediction', color='orange')
plt.show()